https://data-nifc.opendata.arcgis.com/datasets/nifc::wildland-fire-incident-locations/about

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
# load ignition points
ignition_points = pd.read_csv('data/WFIGS_Incident_Locations_CSV.csv')

In [ ]:
ignition_points.columns

In [ ]:
# ig_gdf
ig_gdf = gpd.GeoDataFrame(ignition_points, geometry=gpd.points_from_xy(ignition_points['x'], ignition_points['y']))

In [ ]:
ig_gdf_human_cause = ig_gdf[ig_gdf['FireCause'] == 'Human']
ig_gdf_human_cause.reset_index(drop=True, inplace=True)
# limit ig_gdf_human_cause to only CONUS
ig_gdf_human_cause_conus = ig_gdf_human_cause.cx[-125:-65, 25:50]
# drop values with null IncidentSize
ig_gdf_human_cause_conus.dropna(subset=['IncidentSize'], inplace=True)

In [ ]:
(ig_gdf_human_cause_conus.isnull().sum() / len(ig_gdf_human_cause_conus)) > 0.1

In [ ]:
# drop columns with missing values
ig_gdf_human_cause_conus.dropna(axis=1, inplace=True)

In [ ]:
ig_gdf_human_cause_conus.dtypes

In [ ]:
ig_gdf_human_cause_conus['FireDiscoveryDateTime'] = pd.to_datetime(ig_gdf_human_cause_conus['FireDiscoveryDateTime'])

In [ ]:
ig_gdf_human_cause_conus['year'] = ig_gdf_human_cause_conus['FireDiscoveryDateTime'].dt.year

In [ ]:
ig_gdf_human_cause_conus.sort_values(by='FireDiscoveryDateTime', ascending=False)

In [ ]:
ig_gdf_human_cause_conus.plot.scatter(x='FireDiscoveryDateTime', y='IncidentSize', c='IncidentSize', cmap='viridis', alpha=0.5)

-----

In [ ]:
# create heatmap of human caused fires using folium
import folium
from folium.plugins import HeatMap

m = folium.Map(location=[38, -97], zoom_start=4)

heat_data = [[row['y'], row['x']] for index, row in ig_gdf_human_cause_conus.iterrows()]

HeatMap(heat_data, radius = 15, min_opacity = 0.4, blur = 15, gradient = {0.4: 'blue', 0.65: 'lime', 1: 'red'}, max_zoom=18).add_to(m)

m

In [ ]:
# plot points scaled by IncidentSize using folium
m = folium.Map(location=[38, -97], zoom_start=4)

for i in range(0, len(ig_gdf_human_cause_conus)):
    folium.Circle(
        location=[ig_gdf_human_cause_conus.iloc[i]['y'], ig_gdf_human_cause_conus.iloc[i]['x']],
        radius=np.log10(ig_gdf_human_cause_conus.iloc[i]['IncidentSize'] * 100),
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)

m